# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails

from azureml.interpret import ExplanationClient
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice
from azureml.core import Webservice

import time
import logging

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'HotelBookingDemand'

experiment=Experiment(ws, experiment_name)

In [6]:
cpu_cluster_name = "udacity-casptone-project"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [7]:
dataset = Dataset.get_by_name(ws, name='hotel_booking')
#dataset.to_pandas_dataframe()
training_data, validation_data = dataset.random_split(percentage=0.8, seed=1)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [23]:
# TODO: Put your automl settings here
automl_settings = {
    "name": "AutoML_HotelBookingDemand_Experiment_{0}".format(time.time()),
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 15,
    "primary_metric_name": 'accuracy',
    "max_concurrent_iterations": 5
}

automl_config = AutoMLConfig(task = 'classification',                                                          
                             training_data=training_data,
                             validation_data= validation_data,
                             label_column_name = 'is_canceled',  
                             compute_target=cpu_cluster,                           
                                **automl_settings
                                )                              

In [24]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on udacity-casptone-project with default configuration
Running on remote compute: udacity-casptone-project
Parent Run ID: AutoML_48d3d07c-30cb-4376-98f1-54a3a9a7a960

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

**********************************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [25]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [43]:
# Get your best run and save the model from that run.
best_run, fitted_model = remote_run.get_output(metric = "accuracy")
print(best_run)

description = 'Best AutoML Model'
tags = None

model = remote_run.register_model(description = description, tags = tags)
print(remote_run.model_id)

Run(Experiment: HotelBookingDemand,
Id: AutoML_48d3d07c-30cb-4376-98f1-54a3a9a7a960_23,
Type: azureml.scriptrun,
Status: Completed)
AutoML48d3d07c323


In [29]:
#TODO: Save the best model
print('fitted_model:\n {}\n\n'.format(fitted_model))

fitted_model:
 Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('SparseNormalizer',
                 <azureml.auto...
                                   grow_policy='lossguide', learning_rate=0.1,
                                   max_bin=1023, max_delta_step=0, max_depth=8,
                                   max_leaves=7, min_child_weight=1,
                                   missing=nan, n_estimators=25, n_jobs=1,
                                   nthread=None, objecti

In [30]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
HotelBookingDemand,AutoML_48d3d07c-30cb-4376-98f1-54a3a9a7a960_23,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [44]:
client = ExplanationClient.from_run(best_run)
global_explanation = client.download_model_explanation(top_k=5)
local_importance_values = global_explanation.local_importance_values
expected_values = global_explanation.expected_values

In [45]:
global_explanation_topk = client.download_model_explanation(top_k=5)
global_importance_values = global_explanation_topk.get_ranked_global_values()
global_importance_names = global_explanation_topk.get_ranked_global_names()

In [46]:
print('global importance values: {}'.format(global_importance_values))
print('global importance names: {}'.format(global_importance_names))

global importance values: [2.821015665167897, 0.04699899160267547, 0.020639387197122834, 0.020638391299112258, 0.017426625919431598]
global importance names: ['reservation_status', 'deposit_type', 'country', 'reservation_status_date', 'market_segment']


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [65]:
best_run_metrics = best_run.get_metrics()

for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

mymodel = best_run.register_model(model_name='hoteldemand_automl', model_path='outputs/model.pkl', description = description, tags = tags, properties={'accuracy': best_run_metrics['accuracy']})

weighted_accuracy 1.0
precision_score_weighted 1.0
recall_score_macro 1.0
average_precision_score_macro 1.0
average_precision_score_micro 1.0
precision_score_macro 1.0
f1_score_weighted 1.0
f1_score_macro 1.0
norm_macro_recall 1.0
AUC_micro 1.0
accuracy 1.0
recall_score_weighted 1.0
AUC_macro 1.0
f1_score_micro 1.0
balanced_accuracy 1.0
matthews_correlation 1.0
log_loss 0.04588933788958156
recall_score_micro 1.0
AUC_weighted 1.0
precision_score_micro 1.0
average_precision_score_weighted 1.0
accuracy_table aml://artifactId/ExperimentRun/dcid.AutoML_48d3d07c-30cb-4376-98f1-54a3a9a7a960_23/accuracy_table
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_48d3d07c-30cb-4376-98f1-54a3a9a7a960_23/confusion_matrix


In [ ]:
service_name = "myautomlbestrunv3" # Services Name

myenv = Environment.from_conda_specification(name="env", file_path="myenv.yml")
myenv.register(workspace=ws)

# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py", environment=myenv) # We are going to use score.py from AML

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores =2, memory_gb =4, auth_enabled=True, enable_app_insights=True)


# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(
    workspace = ws,
    name = service_name,
    models = [mymodel],
    inference_config = inference_config,
    deployment_config = deployment_config,
    overwrite=True)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.........................

In [74]:
print(ws.webservices)

service = Webservice(ws, service_name)
print(service.get_logs())

{'myautomlbestrunv2': AciWebservice(workspace=Workspace.create(name='Experiments', subscription_id='c643ea10-d987-475a-a877-13474b748947', resource_group='VioxAI'), name=myautomlbestrunv2, image_id=None, compute_type=None, state=ACI, scoring_uri=None, tags=None, properties={}, created_by={'hasInferenceSchema': 'False', 'hasHttps': 'False'})}
None


In [77]:
print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

primary, secondary = service.get_keys()
print(primary)

Healthy
http://bb0ed55f-217c-4344-abe1-11bf770db3df.eastus.azurecontainer.io/score
http://bb0ed55f-217c-4344-abe1-11bf770db3df.eastus.azurecontainer.io/swagger.json
akq67yM5zoFQsYOgTPV5xNeY9pRhfRoj


TODO: In the cell below, send a request to the web service you deployed to test it.

In [78]:
import requests
import json
import sys

scoring_uri = service.scoring_uri
key = primary

data = {
    "data": [
     {
          "hotel": "Resort Hotel",
          "lead_time": 85,
          "arrival_date_year": 2015,
          "arrival_date_month": "July",
          "arrival_date_week_number": 27,
          "arrival_date_day_of_month": 1,
          "stays_in_weekend_nights": 0,
          "stays_in_week_nights": 3,
          "adults": 2,
          "children": 0,
          "babies": 0,
          "meal": "BB",
          "country": "PRT",
          "market_segment": "Online TA",
          "distribution_channel": "TA/TO",
          "is_repeated_guest": 0,
          "previous_cancellations": 0,
          "previous_bookings_not_canceled": 0,
          "reserved_room_type": "A",
          "assigned_room_type": "A",
          "booking_changes": 0,
          "deposit_type": "No Deposit",
          "agent": "240",
          "company": "NULL",
          "days_in_waiting_list": 0,
          "customer_type": "Trasient",
          "adr": 82,
          "required_car_parking_spaces": 1,
          "total_of_special_requests": 0,
          "reservation_status": "Canceled",
          "reservation_status_date": "2015-05-06 00:00:00"
     }
   ]
}

    
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

headers = {"Content-Type": "application/json"}
    
headers["Authorization"] = f"Bearer {key}"

resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{"result": [1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [80]:
service.delete()